# 1.基础概念

![img](./pic/detr.png)

- 每一层的decoder输出的 100*256都送进FFN进行loss 计算，使得收敛速度更快.
- obj queries除了第一层decoder，其他层都做self attention,让他们知道尽量别搞冗余框，学习 什么地方该抽框。

In [3]:
import torch
from torch import nn
from torchvision.models import resnet50


In [4]:

class DETR(nn.Module):
    def __init__(
        self, num_classes, hidden_dim, nheads, num_encoder_layers, num_decoder_layers
    ):
        super().__init__()
        self.backbone = nn.Sequential(*list(resnet50(weights=True).children())[:-2])
        # 最后两层是AdaptiveAvgPool2d(output_size=(1,1)),Linear(2048,1000)
        self.conv = nn.Conv2d(2048, hidden_dim, 1)
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_decoder_layers, num_encoder_layers
        )  # 定义隐藏层输出维度，头数，encoder与decoder层数。

        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(hidden_dim, 4)
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
        self.row_emb = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_emb = nn.Parameter(torch.rand(50, hidden_dim // 2))

    def forward(self, inputs):
        x = self.backbone(inputs)  # x(1, 2048, 25, 38)经过了32倍缩放。
        h = self.conv(x)
        H, W = h.shape[-2:]
        pos = (
            torch.cat(
                [
                    self.col_emb[:W].unsqueeze(0).repeat(H, 1, 1),
                    self.row_emb[:H].unsqueeze(1).repeat(1, W, 1),
                ],
                dim=-1,
            )
            .flatten(0, 1)
            .unsqueeze(1)
        )
        # HW=950, h(1,256,25,38);pos(950,1,256);self.query_pos(100,256)--(100,1,256)
        h = self.transformer(
            pos + h.flatten(2).permute(2, 0, 1), self.query_pos.unsqueeze(1)
        )
        # 运行结束后h(100,1,256)与送进decoder的query一样
        return self.linear_class(h), self.linear_bbox(h).sigmoid()

detr = DETR(
    num_classes=91, hidden_dim=256, nheads=8, num_encoder_layers=6, num_decoder_layers=6
)


f:\Anaconda\envs\yolo\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


- 对于nn.Transformer,传进去的src为(K,1,h),tgt为(Q,1,h),最终得到的是(Q, 1, h) 与query形状一样。
- 这个1其实是bs,等计算的时候挪到前面,则MHA就是熟悉的(bs,Q,h)与(bs,K,h)了

In [5]:

detr.eval()
inputs = torch.randn(1, 3, 800, 1200)
logits, bbox = detr(inputs)
print(logits.shape, bbox.shape)  # torch.Size([100, 1, 92]) torch.Size([100, 1, 4])

torch.Size([100, 1, 92]) torch.Size([100, 1, 4])


- 由于在encoder里面都是用的MHA，故都不会改变输入的形状，在decoder里面，输入的memory也为(K,1,h)
- .flatten(0, 1)表示将张量从0到1维度展开，比如原始形状为(a,b,c),执行完操作为(ab,c)  
- .flatten(2)表示从第二维度到最后展开，比如原始为(1,a,b,c),执行完操作为(1,a, bc)

## 1.2 补充

**对于nn.Transformer(src, tgt)**  
- 在torch里面封装的nn.Transformer是这样的(右图):对于图像来说，没有mask  
  
<center>

![img](./pic/TRM1.png)
</center>

- 输入进deocer后，经过最后一层encoder layer才输出enc_output.  
- 这个output当作kv送到每一个decoder layer（先对query做self再做cross-attn)中，每个送入的decoder layer只有query是逐个延续的，kv都是原始的enc_output不变。
<center>

![img](./pic/TRM2.png)
</center>
